In [1]:
! pip install Flask


In [8]:
from flask import Flask, request, jsonify
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

# Load data and prepare user-item matrix (as done previously)
ratings = pd.read_csv("C:\\Users\\zahra\\Downloads\\Telegram Desktop\\ratings.csv")
movies = pd.read_csv("C:\\Users\\zahra\\Downloads\\Telegram Desktop\\movies.csv")

merged_data = pd.merge(ratings, movies, on='movieId')
user_item_matrix = merged_data.pivot_table(index='userId', columns='title', values='rating').fillna(0)
# Convert to sparse matrix for better performance
user_item_matrix_sparse = csr_matrix(user_item_matrix.values)

# Compute user similarity
#Compute cosine similarity between users
user_similarity = cosine_similarity(user_item_matrix_sparse)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

# Flask application
app = Flask(__name__)

# Recommendation function (from previous code)
def recommend_movies(user_id, user_item_matrix, user_similarity_df, num_recommendations=5, top_n_similar_users=100):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).head(top_n_similar_users).index
    similar_user_ratings = user_item_matrix.loc[similar_users]
    weighted_ratings = similar_user_ratings.mean().sort_values(ascending=False)
    already_rated = user_item_matrix.loc[user_id]
    recommendations = weighted_ratings[~already_rated.index.isin(already_rated[already_rated > 0].index)]
    recommendations_df = pd.DataFrame(recommendations).reset_index()
    recommendations_df.columns = ['Movie Title', 'Predicted Rating']
    return recommendations_df.head(num_recommendations)

# API route to get recommendations
@app.route('/recommend', methods=['GET'])
def recommend():
    user_id = int(request.args.get('user_id'))
    num_recommendations = int(request.args.get('num_recommendations', 5))
    
    # Call the recommendation function
    recommendations = recommend_movies(user_id=user_id, user_item_matrix=user_item_matrix, user_similarity_df=user_similarity_df, num_recommendations=num_recommendations)
    
    # Convert the recommendations to JSON format
    return recommendations.to_json(orient='records')

# Run the Flask app
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

# Load data and prepare user-item matrix (as done previously)
ratings = pd.read_csv("C:\\Users\\zahra\\Downloads\\Telegram Desktop\\ratings.csv")
movies = pd.read_csv("C:\\Users\\zahra\\Downloads\\Telegram Desktop\\movies.csv")

merged_data = pd.merge(ratings, movies, on='movieId')
user_item_matrix = merged_data.pivot_table(index='userId', columns='title', values='rating').fillna(0)
# Convert to sparse matrix for better performance
user_item_matrix_sparse = csr_matrix(user_item_matrix.values)

# Compute user similarity
#Compute cosine similarity between users
user_similarity = cosine_similarity(user_item_matrix_sparse)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

# Flask application
app = Flask(__name__)

# Recommendation function (from previous code)
def recommend_movies(user_id, user_item_matrix, user_similarity_df, num_recommendations=5, top_n_similar_users=100):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).head(top_n_similar_users).index
    similar_user_ratings = user_item_matrix.loc[similar_users]
    weighted_ratings = similar_user_ratings.mean().sort_values(ascending=False)
    already_rated = user_item_matrix.loc[user_id]
    recommendations = weighted_ratings[~already_rated.index.isin(already_rated[already_rated > 0].index)]
    recommendations_df = pd.DataFrame(recommendations).reset_index()
    recommendations_df.columns = ['Movie Title', 'Predicted Rating']
    return recommendations_df.head(num_recommendations)

# API route to get recommendations
@app.route('/recommend', methods=['GET'])
def recommend():
    user_id = int(request.args.get('user_id'))
    num_recommendations = int(request.args.get('num_recommendations', 5))
    
    # Call the recommendation function
    recommendations = recommend_movies(user_id=user_id, user_item_matrix=user_item_matrix, user_similarity_df=user_similarity_df, num_recommendations=num_recommendations)
    
    # Convert the recommendations to JSON format
    return recommendations.to_json(orient='records')

# Run the Flask app
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [09/Sep/2024 21:01:11] "GET / HTTP/1.1" 404 -
